# Video: https://youtu.be/V4ab6qsJZMY 

## Uso de Sklearn

Estudiemos lo básico sobre como usar la librería Sklearn a través de un ejemplo analizando reseñas de peliculas.

## 1. Leer Dataset

Consideremos el siguiente Dataset:

- Input(x) -> Comentarios (review)
- Ourput(y) -> Sentimientos  

Nuestro objetivo es entrenar un modelo de Machine Learning que permita saber los sentimientos asociados a un review. Con eso en mente, lo primero que debemos hacer es leer el Dataset, para eso podemos usar pandas:

In [1]:
import pandas as pd

df_review = pd.read_csv('IMDB Dataset.csv')
df_review

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


Veamos que este Dataset esta balanceado (es decir, que tiene aproximadamente la misma cantidad de sentimientos positivos y negativos).

In [2]:
df_review.value_counts('sentiment')

sentiment
negative    25000
positive    25000
dtype: int64

Efectivamente lo está, sin embargo, trabajemos un dataset más pequeño desbalanceado con el objetivo de practicar como se balancean los Datasets.

In [3]:
df_positivo = df_review[df_review['sentiment']=='positive'][:9000]
df_negativo = df_review[df_review['sentiment']=='negative'][:1000]

# desbalanceado
df_review_des = pd.concat([df_positivo, df_negativo])
df_review_des.value_counts('sentiment')

sentiment
positive    9000
negative    1000
dtype: int64

## 2. Balancear Dataset

In [4]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler()
df_review_bal, df_review_bal['sentiment'] = rus.fit_resample(df_review_des[['review']],
                                                          df_review_des['sentiment'])

df_review_bal.value_counts(['sentiment'])

sentiment
negative     1000
positive     1000
dtype: int64

Ahora el Dataset balanceado será el que usaremos, es decir: df_review_bal

Teniendo el Dataset adecuado, ahora debemos elegir que datos usaremos para entrenar el modelo y que datos usaremos para testearlo.

## 3. Separar data para entrenar (train) y testear (test)

In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_review_bal, test_size=0.33, random_state=42) #Si test_size = 0.33 entonces el 33% de los datos seran para testear y el 77 % para entrenar.

Ya teniendo los Dataset separados, debemos definirlos en términos de X y Y, habíamos dicho que la x era la entrada (el review), mientras que la y era la salida (el sentimiento):

In [6]:
train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

## 4. Representacion de Text (Bag of Words)

En este caso nuestros datos son texto, es decir, strings, para poder aplicar los algoritmos de Machine learning y que el computador pueda leerlos es necesario convertirlos en texto, para esto hay dos opciones:

### 4.1 Count Vectorizer:

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
text = ["Amo escribir codigo en Python. Amo el código en Python",
        "Odio escribir codigo en Java. Odio el código en Java"]

df = pd.DataFrame({'review': ['review1', 'review2'], 'text':text})
cv = CountVectorizer()
cv_matrix = cv.fit_transform(df['text'])
df_dtm = pd.DataFrame(cv_matrix.toarray(), index=df['review'].values, columns=cv.get_feature_names())
df_dtm

/usr/local/lib64/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,amo,codigo,código,el,en,escribir,java,odio,python
review1,2,1,1,1,2,1,0,0,2
review2,0,1,1,1,2,1,2,2,0


Aquí simplemente se analiza cuantas veces aparece una palabra en cada review. Sin embargo según esto "python" es tan relevante como "en", lo cual no tendría mucho sentido, la otra opción es utilizar:

### 4.2 Tfidf (term frequency - inverse document frequency)

Esta opción analiza la frecuencia en que aparecen las palabras respecto a todo el conjunto de texto.

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
text = ["Amo escribir codigo en Python. Amo el código en Python",
        "Odio escribir codigo en Java. Odio el código en Java"]

df = pd.DataFrame({'review': ['review1', 'review2'], 'text':text})
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['text'])
df_dtm = pd.DataFrame(tfidf_matrix.toarray(), index=df['review'].values, columns=tfidf.get_feature_names())
df_dtm

/usr/local/lib64/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,amo,codigo,código,el,en,escribir,java,odio,python
review1,0.576152,0.204969,0.204969,0.204969,0.409937,0.204969,0.000000,0.000000,0.576152
review2,0.000000,0.204969,0.204969,0.204969,0.409937,0.204969,0.576152,0.576152,0.000000


Así, efectivamente "python" es más importante que "en", como uno esperaría. Dicho esto, apliquemos tfidf a nuestro set de datos:

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(train_x)

test_x_vector = tfidf.transform(test_x) #como ya se hizo fit en la linea de arriba, de ahora en adelante solo hay que hacer transform

In [10]:
train_x_vector

<1340x19752 sparse matrix of type '<class 'numpy.float64'>'
	with 111693 stored elements in Compressed Sparse Row format>

Una matriz sparse es una matriz llena de ceros, en este caso solo 114379 elementos de la matriz son distintos de ceros; sin embargo, la matriz es de 1340 x 20017.

Finalmente, como ya tenemos los datos para entrenar y testear de una manera en que el computador pueda leernos ahora solo faltaría aplicar los modelos de Machine Learning.

## 5. Tipo de modelo de Machine Learning

Existen dos tipos de algoritmos:

- Aprendizaje supervisado (Supervised learning): se conoce cual es el input y cual es el output, la idea es obtener un modelo que para cualquier input genere un output.

- Aprendizaje no supervisado: no se conoce el input ni el output, consiste más en analizar la relación entre todos los datos.

### 5.1 Seleccionar el modelo

En este caso consideraremos un aprendizaje supervizado debido a que el input son los review y el output los sentimientos, ahora solo faltaría saber que algoritmo usar, consideremos varios y los comparamos:

#### 5.1.1 Support Vector Machines (SVM)

In [11]:
from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc.fit(train_x_vector, train_y) #así svc ya estará entrenada y permitira hacer predicciones

SVC(kernel='linear')

Intentemos hacer predicciones con reviews arbitrarias que escribamos:

In [12]:
print(svc.predict(tfidf.transform(['A good movie']))) #buena pelicula
print(svc.predict(tfidf.transform(['An excellent movie']))) #excelente pelicula
print(svc.predict(tfidf.transform(['"I did not like this movie at all I gave this movie away"'])))# no gusto

['negative']
['positive']
['negative']


#### 5.1.2 Decision Tree

In [13]:
from sklearn.tree import DecisionTreeClassifier

dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, train_y)

DecisionTreeClassifier()

#### 5.1.3 Naive Bayes

In [14]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(train_x_vector.toarray(), train_y)

GaussianNB()

#### 5.1.4 Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_x_vector, train_y)

LogisticRegression()

Ahora comparemoslos todos, existen distintas medidas para saber cual algoritmo o modelo es mejor:

### 5.2 Comparación entre distintos modelos

Existen distintas variables para medir cual modelo es mejor, exploremos algunas:

#### 5.2.1 Score (Accuracy)

Devuelve la precisión media en los datos de prueba y las predicciones hechas.

In [16]:
print(svc.score(test_x_vector, test_y))
print(dec_tree.score(test_x_vector, test_y))
print(gnb.score(test_x_vector.toarray(), test_y))
print(lr.score(test_x_vector, test_y))

0.8393939393939394
0.6863636363636364
0.6166666666666667
0.8196969696969697


#### 5.2.2 F1 Score:

F1 Score = 2*(Recall * Precision) / (Recall + Precision)

La puntuación F1 se usa cuando los falsos negativos y los falsos positivos son cruciales de estudiar. Además, F1 tiene en cuenta cómo se distribuyen los datos, por lo que es útil cuando tiene datos con clases de desequilibrio.

In [17]:
from sklearn.metrics import f1_score

f1_score(test_y, svc.predict(test_x_vector),
         labels=['positive', 'negative'],
         average=None)

array([0.84319527, 0.83540373])

#### 5.2.3 Reporte de Clasificacion

In [18]:
from sklearn.metrics import classification_report

print(classification_report(test_y, svc.predict(test_x_vector),
                      labels=['positive', 'negative']))

              precision    recall  f1-score   support

    positive       0.84      0.85      0.84       335
    negative       0.84      0.83      0.84       325

    accuracy                           0.84       660
   macro avg       0.84      0.84      0.84       660
weighted avg       0.84      0.84      0.84       660



#### 5.2.4 Confusion Matrix

Esta permite entender la cantidad de verdaderos verdaderos, verdaderos falsos, falsos verdaderos y falsos falsos, así uno entiende que tan tabien se estan prediciendo los datos: lo ideal es la diagonal sea mucho más grande que las esquinas,

In [19]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, svc.predict(test_x_vector),
                 labels=['positive', 'negative'])

array([[285,  50],
       [ 56, 269]])

Luego de comparar todos estos modelos podemos elegir uno de ellos, ahora lo que se puede hacer es optimizarlo.

### 5.3 Optimizacion del Modelo escogido

Esto no siempre se hace, sin embargo, uno puede darle al modelo algunos parametros de los cuales elegir y este eligira los que mejor le permitan predecir datos:

#### 5.3.1 GridSearchCV

In [20]:
from sklearn.model_selection import GridSearchCV

paremetros = {'C':[1,4,8,16,32], 'kernel':['linear', 'rbf']}
svc = SVC()
svc_grid = GridSearchCV(svc, paremetros, cv=5)
svc_grid.fit(train_x_vector, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 4, 8, 16, 32], 'kernel': ['linear', 'rbf']})

In [21]:
print(svc_grid.best_estimator_)
print(svc_grid.best_params_)

SVC(C=1, kernel='linear')
{'C': 1, 'kernel': 'linear'}


In [22]:
svc_grid.best_score_

0.8395522388059702

## Referencia: https://youtu.be/V4ab6qsJZMY